In [1]:
import pandas as pd
from sqlalchemy import create_engine
from llama_index.llms.openai import OpenAI
from llama_index.core.query_engine import NLSQLTableQueryEngine
from llama_index.core import SQLDatabase
from llama_index.embeddings.openai import OpenAIEmbedding

from llama_index.core.prompts import BasePromptTemplate, PromptTemplate


In [2]:
file_path = "data/out/recipes_flat.json"
df = pd.read_json(file_path)
llm_gpt = OpenAI(model="gpt-4o-mini", temperature=0.5, max_tokens=100)
# embed_gpt = OpenAIEmbedding(model='text-embedding-3-large')


In [3]:
engine = create_engine("sqlite:///data/out/recipes.db")

sql_database = SQLDatabase(
    engine=engine, include_tables=["recipes"], sample_rows_in_table_info=2
)


We need to tune the model because the LLM can return aswers that are not related to the question. We can use a simple heuristic to filter out these answers. We can check if the answer contains any of the words in the question. If it does, we can discard the answer. This heuristic is not perfect, but it can help to improve the performance of the model. we can modify the model for the query search and the response too


In [4]:
PROMPT_MODEL = """
'Given an input question, synthesize a response from the query results.

If the question is about a recipe, provide the recipe details. If the answer is not in the database, respond with "I don't have that information". If the question is not about a recipe, respond with "I just can help you with recipes".


Query: {query_str}
SQL: {sql_query}
SQL Response: {context_str}
Response: '
"""


response_synthesis_prompt = PromptTemplate(
    template=PROMPT_MODEL,
)


In [5]:
query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["recipes"],
    llm=llm_gpt,
    response_synthesis_prompt=response_synthesis_prompt,
)


In [6]:
query_engine.get_prompts()["response_synthesis_prompt"]


PromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['query_str', 'sql_query', 'context_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='\n\'Given an input question, synthesize a response from the query results.\n\nIf the question is about a recipe, provide the recipe details. If the answer is not in the database, respond with "I don\'t have that information". If the question is not about a recipe, respond with "I just can help you with recipes".\n\n\nQuery: {query_str}\nSQL: {sql_query}\nSQL Response: {context_str}\nResponse: \'\n')

In [7]:
query_str = "Dame una receta con pollo"
response = query_engine.query(query_str)
response.response


'Aquí tienes algunas recetas con pollo:\n\n1. **Enchiladas Verdes**\n   - **Ingredientes**:\n     - 1/2 kilos de pechuga de pollo\n     - 1/2 cebolla\n     - 2 chiles serranos\n     - 7 tomates verdes\n     - 1 diente de ajo\n     - 1 rama de cilantro fresco, desinfectado\n     - 1 lechuga orejona\n     - 1 queso fresco'

In [8]:
print(response.metadata["sql_query"])


SELECT name, ingredients, preparation FROM recipes WHERE ingredients LIKE '%pollo%' ORDER BY difficulty


In [9]:
print(response.metadata["result"])


[('Enchiladas Verdes', '1/2 kilos de pechuga de pollo, o maciza de cerdo\n1/2 cebollas\n2 chiles serranos\n7 tomates verdes\n1 diente de ajo\n1 rama de cilantro fresco, desinfectado\n1 lechuga orejona\n1 queso fresco\n16 tortillas\naceite vegetal\n1 pizca de sal\n1/4 tazas de crema ácida', 'Cuece el pollo con la cebolla y la sal en agua hirviendo. Cuando esté cocida deja que enfríe un poco y desmenúzalo.\nEn una olla aparte, cuece los tomates y los chiles serranos durante 5 minutos en agua hirviendo.\nEn la licuadora, licua los tomates hervidos, los chiles serranos, los chiles...'), ('Tabla De Alitas Con 3 Dips', '1 taza de Oikos® Natural Sin Endulzar , 150 g, para el aderezo de cebollín\n1 cucharadita de ajo en polvo, para el aderezo de cebollín\n1 cucharadita de cebolla en polvo, para el aderezo de cebollín\nsal al gusto, para el aderezo de cebollín\npimienta al gusto, para el aderezo de cebollín\n2...', 'Para el dip de cebollín: mezcla el Oikos® Natural Sin Endulzar, el ajo en polvo